In [ ]:
# 1) quasi random numbers
# 2) importance sampling: sample from the region of the tail (look for extremes) that is most important, this will reduce variance 

BASELINE

In [ ]:
import numpy as np, pandas as pd, scipy.stats as st

In [ ]:
data = pd.read_csv('data.csv')
r = np.array([.295, .49, .41, .415, .338, .64, .403, .476])
data['sec_loading'], data['t'] = r[data['sector'].values], st.norm.ppf(data.p)
factors, sample = np.random.normal(0,1, (100_000, len(r)+len(data))), []

for obs in factors:
    m_factor, sec_factor, res_factor = obs[0], obs[:len(r)][data.sector.values], obs[len(r):]
    ind = r[0]**.5 * m_factor + (data.sec_loading-r[0])**.5 * sec_factor + (1-
                                                                        data.sec_loading)**.5 * res_factor < data.t
    loss = np.zeros((len(data),))
    loss[ind] = data[ind].m + data[ind].d * np.random.standard_t(3, size=sum(ind
                                                                         ))
    sample.append(sum(loss))
    

In [ ]:
VaR = sorted(-sample)[100]